In [2]:
from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import math

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split

from scipy import sparse

In [4]:
!wget "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-07-15 02:18:19--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  17.0MB/s    in 0.3s    

2021-07-15 02:18:19 (17.0 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.ba

### 1. Tập dữ liệu
Tập dữ liệu bao gồm 100000 đánh giá của 943 user đối với 1682 bộ phim. Đánh giá (Rating) là các số tự nhiên từ 1 đến 5 thể hiện mức độ hài lòng của user đối với bộ phim đó.
Tập dữ liệu sử dụng trong note book này thuộc bộ dữ liệu MovieLens100k.

```
# Link đường dẫn: https://grouplens.org/datasets/movielens/100k/
```



In [6]:
#Lấy thông tin về User
user_cols = ['id','age','sex','job','zip_code']
users = pd.read_csv('ml-100k/u.user',sep = '|', names = user_cols, encoding = 'latin-1')
users

,id,age,sex,job,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [7]:
#Lấy thông tin về Movie
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item',sep = '|',names = i_cols, encoding = 'latin-1' )
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
#Bộ 100k rating
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


### User_User Collaborative Filtering

In [9]:
# Xây dựng utility Maxtrix 
# movie là hàng, user cột

def utility_matrix(users, items, rating):
  #tao df toan khong
  col = [x for x in users['id']]
  col.insert(0,'movie_id')
  a = np.zeros(shape=(len(items),len(users) + 1))
  df = pd.DataFrame(a,columns = col)

  #thay them user id vào cột user_id
  df['movie_id'] = items['movie id']

  # #xay dung ulity matrix
  for x in tqdm(range(len(rating))):
    user_id = rating.user_id[x]  #do bị lệch 1
    movie_id = rating.movie_id[x] - 1
    rate = rating.rating[x]

    df[user_id][movie_id] = rate

  return df

# chuẩn hóa utility matrix
def normalize_utility(df_1):
  df = df_1.copy()
  df = df.replace(0, np.NaN)

  mean_arr = np.array(df.iloc[:,1:].mean(axis = 0))#mảng mà ở đó vị trí i tưởng ứng với user i+1
  for i in range(len(mean_arr)):
    df[i+1] = df[i+1] - mean_arr[i]
  df = df.replace(np.NaN,0)
  
  return df


In [10]:
df = utility_matrix(users, items, ratings)
df = normalize_utility(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:

def user_user_func(user_id, df, K_neighbor, n_item):
  #loai bo cot movie_id do các gia tri cung hang sẽ thuộc cùng 1 bộ phim nên việc loại bỏ ko  ảnh hưởng
  a = np.array(df.iloc[:,1:])
  neighbor_vectors = np.transpose(a) #chuyen sang dạng vector user(hàng)-movie(cột) để dùng cosine_simi

  #lấy user vector
  user_vector = np.array([neighbor_vectors[user_id-1]])
   
  #tinh similarity cosine 
  similarity = cosine_similarity(neighbor_vectors,user_vector)

  #tao ma tran danh so
  user_ids = np.arange(1,len(neighbor_vectors)+1).reshape(-1,1)

  #tao matran [userid,similarity]
  simi = np.concatenate((user_ids, similarity), axis=1)

  #ranking ma trận simi vừa tạo
  simi_ranked = simi.copy()
  simi_ranked = simi_ranked[simi_ranked[:, 1].argsort()]#sắp xếp theo chiều tang của similarity
  simi_ranked = simi_ranked[::-1]

  
  #ranking san pham [id_item, rating]
  #for trong tung bo phim, check rate của phim nao con thieu
  # for tung user, check user nao da rate phim i roi, đưa vào trong 1 mang
  #loai bo nhung mang nao ko du so ptu de danh gia
  # tính rating từ các ptu trong mang do.

  predicted_rate = []

  for i in tqdm(range(len(user_vector[0]))):
    if user_vector[0,i] == 0:
      k = 0
      nearest_neighbor = []

      for j in range(1, len(simi)): #do vi tri 0 la chinh user do
        neigbor_id = int(simi_ranked[j,0] - 1)
        if neighbor_vectors[neigbor_id,i] != 0:
          nearest_neighbor.append(neigbor_id)
          k+=1
        if k ==  K_neighbor:
          break


      if k == K_neighbor:
        nearest_neighbor = np.array(nearest_neighbor)    
        sum_para = 0
        sum_sim = 0
        for j in range(len(nearest_neighbor)):
            
          sum_para += neighbor_vectors[nearest_neighbor[j], i]*simi[nearest_neighbor[j],1]
          sum_sim += abs(simi[nearest_neighbor[j],1])
        rate = sum_para/sum_sim

        predicted_rate.append([i+1, rate])

  #ranking lại ma trận predict_rate
  predicted_rate = np.array(predicted_rate)
  predicted_rate = predicted_rate [predicted_rate [:, 1].argsort()]
  
  print('Top %s item_id recommended to user_id %s: %s'%\
              (n_item, user_id, str(predicted_rate[-n_item:, 0])))
  # return predicted_rate[::-1]
   

In [23]:
user_id = 10
user_user_func(user_id, df,K_neighbor= 10, n_item=10)


Top 10 item_id recommended to user_id 10: [ 313.  251.  641.  922.  136.  661.  427.  320. 1143.  644.]


array([[ 6.44000000e+02,  1.09143520e+00],
       [ 1.14300000e+03,  1.04418087e+00],
       [ 3.20000000e+02,  1.03769729e+00],
       ...,
       [ 3.68000000e+02, -2.12342352e+00],
       [ 4.57000000e+02, -2.18022576e+00],
       [ 6.88000000e+02, -2.40460289e+00]])

### 2. Item-Item Collaborative Filtering

In [30]:

def item_item_func(user_id, df, n_item):
  #loai bo cot movie_id do các gia tri cung hang sẽ thuộc cùng 1 bộ phim nên việc loại bỏ ko  ảnh hưởng
  neighbor_vectors = np.array(df.iloc[:,1:])

  user_vector = np.transpose(neighbor_vectors)[user_id - 1]

  #Lấy id các item được rate và chưa được rate
  item_rated = [x for x in range(len(user_vector)) if user_vector[x] != 0 ]
  item_not_rated = [x for x in range(len(user_vector)) if user_vector[x] == 0 ]
  
  #tính cosinesimilarity
  similarity = cosine_similarity(neighbor_vectors[item_not_rated], neighbor_vectors[item_rated])

  #loại bỏ những item có sum(similarity) = 0 trong tập id các item chưa rate
  simi_0 = [item_not_rated[x] for x in range(len(similarity)) if sum(similarity[x]) == 0]
  item_not_rated = np.setdiff1d(item_not_rated,simi_0)

  #lấy rating của các item đã được user rate
  user_rating = user_vector[item_rated]
  
  #Dự đoán rate của các sản phẩm
  pred_rate = []
  for i in tqdm(range(len(similarity))):
    if sum(similarity[i]) != 0 : 
      y = sum(similarity[i]*user_rating)/sum(abs(similarity[i]))
      pred_rate.append(y)
    
  #gộp tạo thành mảng [id_item, pred_rate]
  pred_rate = np.array(pred_rate).reshape(-1,1)
  item_not_rated = np.array(item_not_rated).reshape(-1,1)
  pred_item_rating = np.concatenate((item_not_rated, pred_rate), axis=1)

  #ranking và trả về kết quả
  pred_item_rating =pred_item_rating[pred_item_rating[:, 1].argsort()]

  print('Top %s item_id recommended to user_id %s: %s'%\
              (n_item, user_id, str(pred_item_rating[-n_item:, 0])))
  # return pred_item_rating[::-1]
  

In [31]:
user_id = 10

item_item_func(user_id,df, n_item=10)


Top 10 item_id recommended to user_id 10: [ 525.  527.  135.  658. 1612.  643.  632.  986. 1190. 1235.]


### Matrix Factoriztion Collaborative Filtering

In [32]:
class Data(object):
  def __init__(self, dataset, split_rate):
    self.dataset = dataset
    self.split_rate = split_rate
    self.train, self.test = self.split_train_test(self.dataset)

    #do userid, item_id -=1 ở module spli_train_test
    self.n_users = np.max(self.train[:,0] + 1)
    self.n_items = np.max(self.train[:,1] + 1)

    self.Ytrain, self.Rtrain = self.utility_matrix(self.train)
    self.Ytest, self.Rtest = self.utility_matrix(self.test)

    self.Ystad, self.u_mean = self.standardize_Y(self.Ytrain)

    self.n_ratings = self.train.shape[0]

  #tach tập train và test
  def split_train_test(self, dataset):
    train, test = train_test_split(dataset, test_size = self.split_rate, shuffle = True)
    train = np.array(train)
    test = np.array(test)

    # do user_id và item_id bị lệch 1 so với index nên -1 
    train[:,0] -= 1
    train[:,1] -=1

    test[:,0] -=1
    test[:,1] -=1
    
    return train, test

  #Tạo matran thua
  def utility_matrix(self, data_mtx):
    Y = np.zeros(shape = (self.n_items, self.n_users))

    #Matran thua y luu rating của người dùng
    Y = sparse.coo_matrix((data_mtx[:,2], (data_mtx[:,1], data_mtx[:,0])), shape = (self.n_items, self.n_users), dtype = np.float).toarray()
    #Matran thua R luu vi tri duoc rating
    R = sparse.coo_matrix((np.ones((data_mtx.shape[0],)), (data_mtx[:, 1], data_mtx[:, 0])), shape = (self.n_items, self.n_users)).toarray()

    return Y, R

  #chuẩn hóa dữ liệu
  def standardize_Y(self, Y):
    sum_rating = Y.sum(axis = 0)
    u_rating = np.count_nonzero(Y, axis = 0)
    u_mean = sum_rating/u_rating
    for n in range(self.n_users):
      for m in range(self.n_items):
        if Y[m,n] != 0:
          Y[m,n] -= u_mean[n]
    return Y, u_mean


class Model():
  def __init__(self, data, theta, lamda, K):
    self.data = data
    self.theta = theta
    self.lamda = lamda
    self.K = K


    self.I = np.random.randn(data.n_items, K)
    self.U = np.random.randn(K, data.n_users)

  def updateU(self):
    for n in range(self.data.n_users):
      #Lấy các item đã được users rate
      i_rated = np.where(self.data.Ystad[:,n] != 0)[0]

      I_n = self.I[i_rated,:]

      if I_n.shape[0] == 0:#nếu user chưa rate phim nào
        self.U[:,n] = 0
      else:
        s = I_n.shape[0]#số item được rate
        u_n = self.U[:,n]#lấy côt user n trong ma trận U
        y_n = self.data.Ystad[i_rated, n]#Lấy côt user n trong ma trận Y standard

        grad = -1/s*np.dot(I_n.T, (y_n-np.dot(I_n, u_n))) + self.lamda*u_n

        self.U[:,n] -= self.theta*grad

  def updateI(self):
    for m in range(self.data.n_items):
      i_rated = np.where(self.data.Ystad[m,:]!=0)[0]

      U_m = self.U[:,i_rated]#Lấy ra k dòng và i_rated cột
      if U_m.shape[1] == 0:
        self.I[m, :] = 0
      else:
        s = U_m.shape[1]
        i_m = self.I[m, :]
        y_m = self.data.Ystad[m, i_rated]

        grad = -1/s * np.dot(y_m - np.dot(i_m, U_m), U_m.T) + self.lamda*i_m
        self.I[m, :] -= self.theta*grad

  def pred(self, I, U):
    Yhat = np.dot(I,U)

    for n in range(self.data.n_users):
      Yhat[:,n] += self.data.u_mean[n]
    
    Yhat = Yhat.astype(np.int32)

    Yhat[Yhat > 5] = 5
    Yhat[Yhat < 1] = 1

    return Yhat

  #Thay thế những giá trị chưa được rate = 0
  def pred_train_test(self, Yhat, R):

    Y_pred = Yhat.copy()
    Y_pred[R == 0] = 0

    return Y_pred

  def loss(self, Y, Yhat):
    error = Y-Yhat
    n_ratings = np.sum(Y!=0)# đếm số rating
    loss_value = 1/(2*n_ratings)*np.linalg.norm(error, 'fro')**2 + self.lamda/2*(np.linalg.norm(self.I, 'fro')**2 + np.linalg.norm(self.U, 'fro')**2)

    return loss_value

  def RMSE(self, Y, Yhat ):
    error = Y - Yhat
    n_ratings = np.sum(Y != 0)
    rmse = math.sqrt(np.linalg.norm(error, 'fro')**2/n_ratings)
    return rmse

class matrix_factorization():
  def __init__(self, data, model, n_iter, print_log_iter):
    self.data = data
    self.model = model
    self.n_iter = n_iter
    self.print_log_iter = print_log_iter

    self.Y_pred_train = None
    self.Y_pred_test = None
    self.Yhat = None


  def fit(self):
    for i in tqdm(range(self.n_iter)):
      #update U và I
      self.model.updateU()
      self.model.updateI()

      self.Yhat = self.model.pred(self.model.I, self.model.U)

      self.Y_pred_train = self.model.pred_train_test(self.Yhat, self.data.Rtrain)#thay các ptu chưa được rate = 0
      self.Y_pred_test = self.model.pred_train_test(self.Yhat, self.data.Rtest)

      if i % self.print_log_iter == 0:
        print('Iteration: {}; RMSE: {}; Loss value: {}'.
                      format(i, self.model.RMSE(self.data.Ytest, self.Y_pred_test),
                             self.model.loss(self.data.Ytrain, self.Y_pred_train)))

  def recommend_for_user(self, user_id, k_neighbors):
    rec = np.concatenate((np.arange(1,self.Y_pred_test.shape[0]+1).reshape(-1,1), self.Y_pred_test[:, user_id - 1].reshape(-1,1)), axis = 1)

    rec.sort(axis = 0)

    print('Top %s item_id recommended to user_id %s: %s'%\
              (k_neighbors, user_id, str(rec[-k_neighbors:, 0])))



In [35]:
data = Data(dataset =ratings, split_rate = 0.2)
model = Model(data = data, theta = 0.75, lamda = 0.1, K = 3)
mf = matrix_factorization(data, model, 100, 10)
mf.fit()

Iteration: 0; RMSE: 1.2539877871940022; Loss value: 223.22944975046954
Iteration: 10; RMSE: 1.182385161153738; Loss value: 95.92481924381775
Iteration: 20; RMSE: 1.1162920687596745; Loss value: 120.48121360574868
Iteration: 30; RMSE: 1.1096204715188593; Loss value: 126.73974325970036
Iteration: 40; RMSE: 1.1046590483569698; Loss value: 131.02119877646112
Iteration: 50; RMSE: 1.0982184173340932; Loss value: 133.65116449344308
Iteration: 60; RMSE: 1.0940129358711317; Loss value: 134.54308086345267
Iteration: 70; RMSE: 1.0920178736291335; Loss value: 134.75907072530424
Iteration: 80; RMSE: 1.0911075214087815; Loss value: 134.75371268767105
Iteration: 90; RMSE: 1.0902470463432978; Loss value: 134.70146257877673



In [38]:
#recommend
mf.recommend_for_user(user_id=300, k_neighbors=5)

Top 5 item_id recommended to user_id 300: [1678 1679 1680 1681 1682]
